### 데이터 불러오기

In [10]:
import pandas as pd

df_tr = pd.read_csv("./train_v1.0.2_clean_spacing.csv")
df_v = pd.read_csv("../raw/dev.csv")
df_tt = pd.read_csv("../raw/test.csv")

In [11]:
df_tr.sample(5)

,id,source,sentence_1,sentence_2,label,binary-label
1808,boostcamp-sts-v1-train-1808,nsmc-sampled,돈 내고 보기 아까운 영화 네요..,오래 만에 감동을 불러 일으키는 영화 네요..,0.6,0.0
3200,boostcamp-sts-v1-train-3200,petition-sampled,미세먼지 많이 마시고 수술 받은 사람입니다,추천 누르시고 이명박이랑 미세먼지 동의 해주세요,0.6,0.0
5260,boostcamp-sts-v1-train-5260,nsmc-sampled,그분은 항상 제 편을 들어주셨었습니다.,아버지도 제 편이 아니었고 어머니도 같이 있지 않을 때 끝까지 제 편이었던 선생님....,1.0,0.0
123,boostcamp-sts-v1-train-123,petition-sampled,집값 안정화에 대한 청원,집값 안정화를 위한 제안,3.2,1.0
7,boostcamp-sts-v1-train-007,nsmc-sampled,이렇게 귀여운 쥐들은 처음이네요. ㅎㅎㅎ,이렇게 지 겨운 공포영화는 처음..,0.6,0.0


### sentence 정제

In [3]:
import re
from pykospacing import Spacing

spacing = Spacing()


def clean_text(text):

    # 문장 속 / 삭제
    text = re.sub(r"/", " ", text)

    # 문장 속 ㅋ가 4개 이상 일 경우 ㅋㅋㅋ으로 변경
    text = re.sub(r"ㅋ{4,}", "ㅋㅋㅋ", text)

    # 문장 속 ㅎ가 3개 이상인 경우 ㅎ로 변경
    text = re.sub(r"ㅎ{4,}", "ㅎㅎㅎ", text)

    # 문장 속 ~가 1개 이상인 경우 ~로 변경
    text = re.sub(r"~{2,}", "~", text)

    # 문장 속 ;가 3개 이상인 경우 ;;로 변경
    text = re.sub(r";{3,}", ";;", text)

    # 문장 속 .가 3개 이상인 경우 ...로 변경
    text = re.sub(r"\.{4,}", "...", text)

    # 문장 속 ,가 3개 이상인 경우 ,,,로 변경
    text = re.sub(r"\,{4,}", ",,,", text)

    # 문장 속 !가 3개 이상인 경우 !!!로 변경
    text = re.sub(r"!{4,}", "!!!", text)

    # 문장 속 ?가 3개 이상인 경우 ???로 변경
    text = re.sub(r"\?{4,}", "???", text)

    # 문장 속 ^가 3개 이상인 경우 ^^로 변경
    text = re.sub(r"\^{3,}", "^^", text)

    # 문장 속 ㅠ가 3개 이상인 경우 ㅠㅠ로 변경
    text = re.sub(r"ㅠ{3,}", "ㅠㅠ", text)

    # 문장 속 ㅜ가 3개 이상인 경우 ㅜㅜ로 변경
    text = re.sub(r"ㅜ{3,}", "ㅠㅠ", text)

    return text


def processing(df):
    df["sentence_1"] = df["sentence_1"].apply(spacing)
    df["sentence_2"] = df["sentence_2"].apply(spacing)

    # 문장 변경
    df["sentence_1"] = df["sentence_1"].apply(clean_text)
    df["sentence_2"] = df["sentence_2"].apply(clean_text)

    if "sentence_1_clean" in df.columns:
        df.drop(columns=["sentence_1_clean"], inplace=True)

    if "sentence_2_clean" in df.columns:
        df.drop(columns=["sentence_2_clean"], inplace=True)

    return df

In [4]:
df_tr = processing(df_tr)
df_v = processing(df_v)
df_tt = processing(df_tt)

### 문맥을 고려한 텍스트 데이터 증강

In [17]:
import transformers
import re
import random
import numpy as np


class BERT_Augmentation:
    def __init__(self):
        self.model_name = "monologg/koelectra-base-v3-generator"
        self.model = transformers.AutoModelForMaskedLM.from_pretrained(self.model_name)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(self.model_name)
        self.unmasker = transformers.pipeline(
            "fill-mask", model=self.model, tokenizer=self.tokenizer, device=0
        )
        random.seed(42)

    # 랜덤하게 토큰? 글자에 마스크 씌우기
    def random_masking_replacement(self, sentence, ratio=0.15):
        """Masking random eojeol of the sentence, and recover them using PLM.

        Args:|
            sentence (str): Source sentence
            ratio (int): Ratio of masking

        Returns:
          str: Recovered sentence
        """

        span = int(round(len(sentence.split()) * ratio))

        # 품질 유지를 위해, 문장의 어절 수가 4 이하라면 원문장을 그대로 리턴합니다.
        if len(sentence.split()) <= 4:
            return sentence

        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        unmask_sentence = sentence
        # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
        random_idx = random.randint(1, len(unmask_sentence.split()) - span)

        unmask_sentence = unmask_sentence.split()
        # del unmask_sentence[random_idx:random_idx+span]
        cache = []
        for _ in range(span):
            # 처음과 끝 부분을 [MASK]로 치환 후 추론할 때의 품질이 좋지 않음.
            while cache and random_idx in cache:
                random_idx = random.randint(1, len(unmask_sentence) - 2)
            cache.append(random_idx)
            unmask_sentence[random_idx] = mask
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]["sequence"]
            unmask_sentence = unmask_sentence.split()
        unmask_sentence = " ".join(unmask_sentence)
        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

    def random_masking_insertion(self, sentence, ratio=0.15):

        span = int(round(len(sentence.split()) * ratio))
        mask = self.tokenizer.mask_token
        unmasker = self.unmasker

        # Recover
        unmask_sentence = sentence

        for _ in range(span):
            unmask_sentence = unmask_sentence.split()
            random_idx = random.randint(0, len(unmask_sentence) - 1)
            unmask_sentence.insert(random_idx, mask)
            unmask_sentence = unmasker(" ".join(unmask_sentence))[0]["sequence"]

        unmask_sentence = unmask_sentence.replace("  ", " ")

        return unmask_sentence.strip()

In [18]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import random
import requests
from kiwipiepy import Kiwi
import time
from quickspacer import Spacer


class AdverbAugmentation:
    def __init__(self):
        self.kiwi = Kiwi()
        self.spacing = Spacer().space
        self.gloss_cache = {}  # 캐시 추가

    def _adverb_detector(self, sentence):

        # POS info
        pos_list = [(x[0], x[1]) for x in self.kiwi.tokenize(sentence)]  # (token, pos)

        adverb_list = []
        for pos in pos_list:
            if pos[1] == "MAG" and len(pos[0]) > 1:  # 1음절 부사는 제외함.
                adverb_list.append(pos[0])
        return adverb_list

    def _get_gloss(self, word):
        if word in self.gloss_cache:  # 캐시 확인
            return self.gloss_cache[word]

        res = requests.get("https://dic.daum.net/search.do?q=" + word, timeout=5)
        time.sleep(random.uniform(0.5, 2.5))
        soup = BeautifulSoup(res.content, "html.parser")
        try:
            # 첫 번째 뜻풀이.
            meaning = soup.find("span", class_="txt_search")
        except AttributeError:
            return word
        if not meaning:
            return word

        # parsing 결과에서 한글만 추출
        meaning = re.findall("[가-힣]+", str(meaning))
        meaning = " ".join(meaning)

        # 띄어쓰기 오류 교정 (위 에 -> 위에)
        # meaning = spell_checker.check(meaning).as_dict()['checked'].strip()
        meaning = self.spacing([meaning.replace(" ", "")])

        self.gloss_cache[word] = meaning[0].strip()  # 캐시에 저장

        return meaning[0].strip()

    def adverb_gloss_replacement(self, sentence):
        adverb_list = self._adverb_detector(sentence)
        if adverb_list:
            # 부사들 중에서 1개만 랜덤으로 선택합니다.
            adverb = random.choice(adverb_list)
            try:
                gloss = self._get_gloss(adverb)
                sentence = sentence.replace(adverb, gloss)
            except:
                print("except: ", sentence)
                pass
        return sentence

In [19]:
import random
import pickle

wordnet = {}
with open("./wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)


def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    if len(new_words) != 0:
        sentence = " ".join(new_words)
        new_words = sentence.split(" ")

    else:
        new_words = ""

    return new_words


def get_synonyms(word):
    synomyms = []

    try:
        for syn in wordnet[word]:
            for s in syn:
                synomyms.append(s)
    except:
        pass

    return synomyms

In [20]:
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
import asyncio


# 데이터 로드
df = df_tr

original_data = df.copy()

# BERT 증강 및 부사 치환 증강 클래스 인스턴스 생성
BERT_aug = BERT_Augmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion
adverb_aug = AdverbAugmentation()
adverb_gloss_replacement = adverb_aug.adverb_gloss_replacement


# 증강 함수 정의
def augment_sentence(row, aug_method):
    row["sentence_1"] = aug_method(row["sentence_1"])
    row["sentence_2"] = aug_method(row["sentence_2"])
    return row


# 각 증강 기법에 따라 데이터셋 증강
def augment_dataset(df, aug_method, source_label, n_jobs=1, num_aug=1):
    augmented_rows = []
    for _ in range(num_aug):  # 각 문장에 대해 num_aug 만큼 증강
        augmented_rows += Parallel(n_jobs=n_jobs)(
            delayed(augment_sentence)(row.copy(), aug_method)
            for idx, row in tqdm(df.iterrows(), total=len(df))
        )
    augmented_df = pd.DataFrame(augmented_rows)
    augmented_df["source"] = source_label  # 증강 방법에 따라 source 설정
    return augmented_df


# 원본 라벨 유지
original_data["source"] = "original"

# 증강 기법 적용

# 1. Random Masking Insertion (BERT)
print("Random Masking Insertion 중...")
masking_insertion_augset = augment_dataset(
    df.copy(), lambda x: random_masking_insertion(x, ratio=0.15), "data-argu-RMI"
)

# 2. 부사 치환
print("부사 치환 중...")
adverb_augset = augment_dataset(df.copy(), adverb_gloss_replacement, "data-argu-adverb")

# 원본 데이터와 결합 및 중복 제거
augmented_data = pd.concat([masking_insertion_augset, adverb_augset])
augmented_data = augmented_data.drop_duplicates(
    subset=["sentence_1", "sentence_2"]
).reset_index(drop=True)

# 증강 데이터에 id 업데이트
augmented_data["id"] = ["argu-{:03d}".format(i) for i in range(len(augmented_data))]

# 원본 데이터와 결합 및 중복 제거
final_data = (
    pd.concat([original_data, augmented_data])
    .drop_duplicates(subset=["sentence_1", "sentence_2"])
    .reset_index(drop=True)
)

# 결과 저장
final_data.to_csv("./test_argu.csv", index=False)

Random Masking Insertion 중...


100%|██████████| 9324/9324 [03:06<00:00, 50.01it/s]


부사 치환 중...


  6%|▌         | 542/9324 [03:59<28:46,  5.09it/s]  

except:  순진+어리버리 연기를 했던 이완.. 짱 귀여움


...

 78%|███████▊  | 7254/9324 [13:49<01:57, 17.61it/s]

except:  내일 무료체험은 오후 2시입니다~


 80%|███████▉  | 7442/9324 [13:57<01:42, 18.42it/s]

except:  청년 내일채움공제의 신청 자격을 확대해 주세요.


100%|██████████| 9324/9324 [15:06<00:00, 10.28it/s]
